In [ ]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
import segyio
import gstools as gs
import os
import lasio

In [ ]:
from scipy.interpolate import interp1d, interp2d
from scipy.interpolate import griddata, RegularGridInterpolator, LinearNDInterpolator

In [ ]:
import numpy as np
from scipy.ndimage import map_coordinates
from scipy import signal
from scipy.interpolate import interp1d
import gstools as gs
from scipy.interpolate import RectBivariateSpline

def find_normalized_thickness(surfs, loc):
    """
    Compute the normalized cumulative thickness between surfaces.
    Parameters
    ----------
    surfs : list of ndarray
        List of 2D arrays representing surfaces.
    loc : tuple of int
        Location in inline and crossline indices (i_il, i_xl).
    Returns
    -------
    ndarray
        Cumulative normalized thickness.
    """
    i_il, i_xl = loc
    thick = np.zeros(len(surfs))
    for i in range(1, len(surfs)):
        thick[i] = abs(surfs[i][i_il, i_xl] - surfs[i - 1][i_il, i_xl])
    thick = abs(thick) / np.sum(thick)
    return np.cumsum(thick)

def deflat(cube, ijk, silence=True):
    """
    Deflat a data cube using precomputed indices.
    Parameters
    ----------
    cube : ndarray
        3D seismic data array (il,xl,nsamples).
    ijk : ndarray
        3D index map for deflattening (il,xl,nsamples).
    silence : bool, optional
        If True, suppress progress messages. Default is True.
    Returns
    -------
    ndarray
        Deflattened cube.
    """
    new_cube = np.empty_like(ijk)
    for i in range(cube.shape[0]):
        cube_cut = cube[i].T.copy()
        ijk_cut = ijk[i].T.copy()
        _, x = np.indices(ijk_cut.shape)
        y = ijk_cut * (cube.shape[2] - 1)
        y[:, -1] = y[:, -2]
        deformed = map_coordinates(cube_cut, (y, x))
        new_cube[i] = deformed.T
        if not silence:
            print(f"Deflattening Inline Index [{i}] {((i + 1)/ cube.shape[0]) * 100:.2f}%\r", end="")
    return new_cube

def low_pass(cube, cut_hz_vert, dt_s, cut_hz_hor=15, silence=True):
    """
    Apply a low-pass filter to a data cube.
    Parameters
    ----------
    cube : ndarray
        3D seismic data array (il,xl,nsamples).
    cut_hz : float
        Cut-off frequency in Hz.
    dt_s : float
        Sampling interval in seconds.
    silence : bool, optional
        If True, suppress progress messages. Default is True.
    Returns
    -------
    ndarray
        Filtered seismic cube.
    """
    nfilt = 1
    samplerate_hz = 1 / dt_s
    nyquist = samplerate_hz / 2
    b, a = signal.butter(nfilt, cut_hz_vert / nyquist, btype='lowpass')
    b2, a2 = signal.butter(nfilt, cut_hz_hor / nyquist, btype='lowpass')
    new_cube = np.empty_like(cube)
    for i in range(cube.shape[0]):
        new_cube[i, :, :] = signal.filtfilt(b, a, cube[i, :, :], axis=0)
        new_cube[i, :, :] = signal.filtfilt(b2, a2, cube[i, :, :], axis=1)
        if not silence:
            print(f"Filtering Inline Index [{i}] {((i + 1)/ cube.shape[0]) * 100:.2f}%\r", end="")
    for j in range(cube.shape[1]):
        new_cube[:, j, :] = signal.filtfilt(b2, a2, cube[:, j, :], axis=1)
        if not silence:
            print(f"Filtering Crossline Index [{j}] {((j + 1)/ cube.shape[1]) * 100:.2f}%\r", end="")
    return new_cube

class BMTools:
    """
    Class for handling a full background modeling workflow.
    """
    def __init__(self):
        """Initialize BMTools with default attributes."""
        self.logs = None            # Well logs
        self.pos = None             # Relative well locations
        self.seismic_shape = None   # Shape of the seismic cube
        self.rgt_thick = None       # Relative Geological Time thickness
        self.rgt = None             # RGT cube
        self.logs_flat = None       # Flattened logs
        self.krig_flat = None            # Kriging results
        self.krig = None        # Deformed cube
        self.time = None            # Time vector
        self.dt = None              # Time sampling interval
        self.dt_s = None            # Time sampling in seconds
        self.rgt_ref = None         # Reference RGT trace
        self.model = None        # Low-pass filtered data (background model)

    def build_rgt(self, surfaces, time, silence=True, ref_trace=None):
        """
        Build Relative Geological Time (RGT) volume.
        Parameters
        ----------
        surfaces : ndarray
            3D array of interpreted surfaces (il,xl,nsamples).
        time : ndarray
            1D array of time values in milliseconds (nsamples).
        silence : bool, optional
            If True, suppress progress messages. Default is True.
        Returns
        -------
        BMTools
            Self instance with updated RGT attributes.
        """
        self.seismic_shape = np.asarray((surfaces.shape[1], surfaces.shape[2], len(time)))
        if not ref_trace:
            ref_trace = (self.seismic_shape[0]//2, self.seismic_shape[1]//2)
        self.rgt_thick = find_normalized_thickness(surfaces, ref_trace)
        self.rgt = np.full(self.seismic_shape, np.nan)
        self.time = time.copy()
        self.dt = self.time[1] - self.time[0]
        self.dt_s = self.dt / 1000
        nlayers = len(surfaces) - 1

        for i in range(self.seismic_shape[0]):
            for j in range(self.seismic_shape[1]):
                for k in range(nlayers):
                    mask = np.arange(
                        (surfaces[k][i, j] - self.time[0]) // self.dt,
                        (surfaces[k + 1][i, j] - self.time[0]) // self.dt
                    ).astype(int)
                    self.rgt[i, j, mask] = np.linspace(self.rgt_thick[k], self.rgt_thick[k + 1], len(mask))
                self.rgt[i, j, :np.nanargmin(self.rgt[i, j])] = 0
                self.rgt[i, j, np.nanargmax(self.rgt[i, j]):] = 1
            if not silence:
                print(f"Building RGT Model Inline Index [{i}] ({((i + 1)/ self.seismic_shape[0]) * 100:.2f})%\r", end="")
        return self

    def read_wells(self, well_logs, relative_well_loc, rgt_domain_length=500):
        """
        Process well logs and map to RGT.
        Parameters
        ----------
        well_logs : ndarray
            2D array of well logs (nwells, nsamples).
        relative_well_loc : ndarray
            2D array of well locations (index of inline, index of crossline).
        Returns
        -------
        BMTools
            Self instance with updated well log attributes.
        """
        self.logs_z = well_logs
        self.pos = relative_well_loc.T
        self.logs_flat = np.empty((self.logs_z.shape[0], rgt_domain_length))
        self.rgt_ref = np.linspace(0,1,rgt_domain_length)

        for i in range(self.logs_flat.shape[0]):
            il_index, xl_index = self.pos
            index_rgt = self.rgt[il_index[i], xl_index[i], :].copy()
            index_rgt[:np.nanargmin(index_rgt)] = 0
            index_rgt[np.nanargmax(index_rgt) + 1:] = 1
            interpolator = interp1d(index_rgt, self.logs_z[i], fill_value=np.nan, kind="linear")
            self.logs_flat[i, :] = interpolator(self.rgt_ref)
        return self

    def interpol(self, variogram, well_logs, relative_well_loc, fill_value, skip_values=0, second_var=None, silence=True, decimate=0, rgt_domain_length=500):
        """
        Perform kriging, deflattening and low-pass filter.
        Parameters
        ----------
        variogram : gstools.Variogram
            Variogram (gsstools) model for kriging.
        cut_hz : float
            Cut-off frequency in Hz for low-pass filtering.
        fill_value : tuple of float
            Values to fill in top and base regions.
        silence : bool, optional
            If True, suppress progress messages. Default is True.
        Returns
        -------
        BMTools
            Self instance with updated attributes.
        """
        self.read_wells(well_logs, relative_well_loc, rgt_domain_length)
        self.rgt_ref = np.linspace(0, 1, rgt_domain_length)
        self.krig_flat = np.zeros((self.rgt.shape[0], self.rgt.shape[1], rgt_domain_length))
        self.krig = np.zeros_like(self.rgt)
        gridy = np.arange(self.seismic_shape[0], dtype=np.float32)
        gridx = np.arange(self.seismic_shape[1], dtype=np.float32)
        x, y = (self.pos[1], self.pos[0])
        top, base = True, False
        if decimate>0:
            gridx_dec = gridx.copy()[::decimate]
            gridy_dec = gridy.copy()[::decimate]

        for i in range(self.rgt_ref.shape[0]):
            if self.rgt_ref[i] == 0:
                self.krig_flat[:, :, i] = fill_value[0]
            elif self.rgt_ref[i] == 1:
                self.krig_flat[:, :, i] = fill_value[1]
            else:
                dvals = self.logs_flat[:, i].copy()
                #dvals[dvals==skip_values] = np.nan
                if np.sum(~np.isnan(dvals)) > 0:
                    OK = gs.krige.ExtDrift(variogram, (x, y), dvals, ext_drift = second_var[self.pos[0][~np.isnan(dvals)],
                                                                                            self.pos[1][~np.isnan(dvals)],
                                                                                            i])
                    if decimate<=0:
                        self.krig_flat[:, :, i] = OK.structured([gridx, gridy], ext_drift=second_var[:,:,i])[0].T
                    else:
                        krig_dec = OK.structured([gridx_dec, gridy_dec], ext_drift=second_var[::decimate,::decimate,i])[0]
                        interpol = RectBivariateSpline(gridx_dec, gridy_dec, krig_dec)
                        OK_structured = interpol(gridx, gridy)
                        self.krig_flat[:, :, i] = OK_structured.T
                    top, base = False, True
                elif top:
                    self.krig_flat[:, :, i] = fill_value[0]
                elif base:
                    self.krig_flat[:, :, i] = fill_value[1]
            if not silence:
                print(f"Kriging Index [{i}] ({((i + 1) / self.seismic_shape[2]) * 100:.2f}%) done\r", end="")
        self.krig = deflat(self.krig_flat, self.rgt, silence)
        return self

    def smooth_model(self, cut_hz_vert, cut_hz_hor=15, silence=True):
        self.model = low_pass(self.krig, cut_hz_vert, self.dt_s, cut_hz_hor, silence)
        return self

In [ ]:
#Loading seismic using Segyio lib

# string containing the path location of the seismic data at disk
velocity_flat = np.load('Velocity_Model_Time_FLAT.npy' )

In [ ]:
seis = segyio.open(r'Velocity_Model_Time.sgy', iline=189, xline=193)
velocity = segyio.cube(seis)
velocity = velocity[:-1,:-1,425:1025]
velocity.shape

In [ ]:
import os

# Define o caminho para a pasta contendo os arquivos LAS
pasta_las = 'Wells/Wells/'

# Dicionário para armazenar os valores de skiprows para cada poço
skiprows_pocos = {}

# Loop pelos arquivos LAS na pasta para encontrar skiprows
for arquivo in os.listdir(pasta_las):
    if arquivo.endswith('.las'):
        nome_poco = arquivo.split('.')[0]

        with open(os.path.join(pasta_las, arquivo), 'r') as f:
            for i, line in enumerate(f):
                if '~	DEPTH' in line:  # Substitua '~A' pela linha que indica o início dos dados
                    skiprows_pocos[nome_poco] = i + 1
                    break
            else:
                skiprows_pocos[nome_poco] = 0  # Valor padrão se a linha não for encontrada

# Cria um dicionário para armazenar os DataFrames de cada poço
dados_pocos = {}

# Loop pelos arquivos LAS na pasta para ler os dados
for arquivo in os.listdir(pasta_las):
    if arquivo.endswith('.las'):
        nome_poco = arquivo.split('.')[0]
        skiprows = skiprows_pocos[nome_poco]  # Obtém o skiprows do dicionário

        # Lê o arquivo LAS e define os nomes das colunas
        names = pd.read_csv(os.path.join(pasta_las, arquivo), sep='\s+', skiprows=skiprows - 1, nrows=0)
        names = names.columns[1:]

        # Lê o arquivo LAS com os dados
        print(nome_poco)
        df = pd.read_csv(os.path.join(pasta_las, arquivo), sep='\s+', skiprows=skiprows, na_values=-9999,
                         usecols=np.arange(len(names)), names=names)

        # Check if 'VP' column exists before calculating 'IS'
        df['CALIR'] = np.nan
        if 'DTCO' in df.columns:
            df['VP'] = 304800 / df['DTCO']
            df['IP'] = df['VP']*df['RHOB']

        else:
            # Handle the case where 'VP' is missing, e.g., set 'IS' to NaN
            df['VP'] = np.nan
            df['IP'] = np.nan

        # Check if 'VS' column exists before calculating 'IS'
        if 'DTS' in df.columns:
            df['VS'] = 304800 / df['DTS']
            df['IS'] = df['VS'] * df['RHOB']
        else:
            # Handle the case where 'VS' is missing, e.g., set 'IS' to NaN
            df['IS'] = np.nan


        # Adiciona uma coluna com o nome do poço
        df['Well'] = nome_poco

        # Armazena o DataFrame no dicionário
        dados_pocos[nome_poco] = df

# Concatena todos os DataFrames em um único DataFrame
df_final = pd.concat(dados_pocos.values(), ignore_index=True)
df_final = df_final[['Well','DEPTH','IP']]

In [ ]:
dt_ms = 4

## Horizontes

Carregando os horizontes a partir de arquivos .npy. Porem, no Andromeda, serao utilizados os horizontes importados no projeto

In [ ]:
surf_saltbase = np.load(os.path.join(r'E:/db-horizons/SaltBase_v1_TIME_grid.npy')).T
surf_intraalagoas = np.load(os.path.join(r'E:/db-horizons/IntraAlagoas_v0_TIME_grid.npy')).T
surf_prealagoas = np.load(os.path.join(r'E:/db-horizons/PreAlagoas_v1_TIME_grid.npy')).T
surf_intrajiquia = np.load(os.path.join(r'E:/db-horizons/IntraJiquia_v1_TIME_grid.npy')).T
surf_picarras = np.load(os.path.join(r'E:/db-horizons/PreJiquia_V0_TIME_grid.npy')).T

In [ ]:
hrz_ma = pd.read_csv('Horizontes/Time/Marco_Azul.txt', sep='\s+', usecols=(2,5,8), names=['IL','XL','TWT']).sort_values(by=['IL', 'XL'])
hrz_base140 = pd.read_csv('Horizontes/Time/140_Base.txt', sep='\s+', usecols=(2,5,8), names=['IL','XL','TWT']).sort_values(by=['IL', 'XL'])
hrz_mio = pd.read_csv('Horizontes/Time/Mioceno.txt', sep='\s+', usecols=(2,5,8), names=['IL','XL','TWT']).sort_values(by=['IL', "XL"])

### Grid

In [ ]:
il = np.arange(hrz_ma.IL.min(), hrz_ma.IL.max())
xl = np.arange(hrz_ma.XL.min(), hrz_ma.XL.max())
il, xl

In [ ]:
def grid_surface_as_seismic(df, il, xl):
    x, y, z = df[['IL', 'XL', 'TWT']].dropna().values.T
    
    valid_indices = ~np.isnan(x) & ~np.isnan(y) & ~np.isnan(z) # Eliminate any NaN
    x, y, z= x[valid_indices], y[valid_indices], z[valid_indices]
    
    xx, yy = np.meshgrid(il, xl) # Create a meshgrid
    
    xi = np.column_stack((xx.ravel(), yy.ravel())) # Reshape meshgrid into a (n_points, 2) array
    
    grid = griddata((x, y), z, xi, method='linear', fill_value=np.mean(z)).reshape(xx.shape) # Perform interpolation
    return grid.T

In [ ]:
hrz_ma_grid = grid_surface_as_seismic(hrz_ma, il, xl)
hrz_base140_grid = grid_surface_as_seismic(hrz_base140, il, xl)
hrz_mio_grid = grid_surface_as_seismic(hrz_mio, il, xl)

## Perfis

### Leitura de TDR

In [ ]:
df_final['TWT'] = np.nan
df_final['TVDSS'] = df_final['DEPTH'] - 26.5
for file in os.listdir('TDR/'):
    dfi = lasio.read(os.path.join('TDR/',file)).df().reset_index()
    wellname = file.replace('_TDR','-RJS')
    interpolador = interp1d(dfi['TVDSS'], dfi['ETIM'], fill_value='extrapolate', bounds_error=False)
    mask = (df_final.Well == wellname)
    df_final.loc[mask, 'TWT'] = interpolador(df_final[mask].TVDSS)
    print(file, dfi.columns, wellname)

### Reamostrar para 4ms

Considere uma matriz com os poços empilhados um por linha contendo a mesmo tamanho e amostrado na mesma dimensão do modelo, ou seja: (nwells,nsamples)

In [ ]:
time = np.arange(1700,4100,4)
len(df_final.Well.unique()), len(time)

In [ ]:
log_well = np.zeros((16,600))
i = 0
df_final['IP_ups'] = df_final['IP']
for well in df_final.Well.unique():
    mask = (df_final.Well == well)
    df_final.loc[mask,'IP_ups'] = np.convolve(np.ones(30)/30, df_final[mask].IP, mode='same')
    interpolador = interp1d(df_final[mask].TWT, df_final[mask].IP_ups, bounds_error=False, fill_value='extrapolate')
    log_well[i,:] = interpolador(time)
    i += 1

In [ ]:
wells_names = [
'1-RJS-342',
'3-RJS-355',
'3-RJS-360A',
'3-RJS-510A',
'4-ABL-30A',
'4-RJS-367',
'4-RJS-477A',
'6-ABL-1',
'9-AB-67',
'9-AB-71',
'9-ABL-2',
'9-ABL-3B',
'9-ABL-5',
'9-ABL-6A',
'9-ABL-7',
 '9-ABL-9D']

#List pf wells positions in order
positions = [
[1118, 1410],
[1387, 1347],
[1220, 1703],
[940, 2376],
[1318, 2523],
[906, 2183],
[889, 1584],
[1304, 2088],
[1107, 1493],
[1054, 1500],
[1359, 1694],
[1017, 1778],
[1347, 1816],
[1466, 1823],
[1190, 1780],
[1071, 2031]]
positions = np.asarray(positions)


In [ ]:
relative_well_pos = positions.copy()
relative_well_pos[:,0] = relative_well_pos[:,0] - il.min()
relative_well_pos[:,1] = relative_well_pos[:,1] - xl.min()
relative_well_pos

## Inserir Horizontes e construir RGT

In [ ]:
# shift na base do sal pra cima (limite superior do modelo)
hrz_mio_grid_shift = hrz_mio_grid - 60

#shift na picarras pra baixo (limite inferior do modelo)
hrz_ma_grid_shift = hrz_ma_grid + 60

# superficies em ordem da rasa para a mais profunda
surfs = np.asarray([hrz_mio_grid_shift,
                   hrz_mio_grid,
                   hrz_base140_grid,
                   hrz_ma_grid,
                   hrz_ma_grid_shift])

surfs.shape

### Construir o Modelo

In [ ]:
mymodel = BMTools()
mymodel.build_rgt(surfs, time, silence=False)
mymodel.rgt.shape

### Modelagem

In [ ]:
variogram = gs.Spherical(dim=2, len_scale=200, var=np.nanvar(log_well))
angle = - np.pi / 8 # (-22.5º)
variogram = gs.Gaussian(dim=2, len_scale=[200, 100], angles=angle, var=np.nanvar(log_well))

# Executar a interpolacao (Krigagem ordinaria)
mymodel.interpol(
    variogram, 
    log_well, 
    relative_well_pos, 
    fill_value=(3000,6000), 
    silence=False, 
    decimate=40, 
    second_var = velocity_flat
    )

In [ ]:
mapa = np.zeros_like(hrz_base140_grid)
for i in range(hrz_base140_grid.shape[0]):
    for j in range(hrz_base140_grid.shape[1]):
        z = int((hrz_base140_grid[i,j]-1700)//4)
        #mapa[i,j] = mymodel.krig[i,j,z]
        mapa[i,j] = velocity[i,j,z]

### Filtro passa-baixa

In [ ]:
mymodel.smooth_model(cut_hz=50, silence=False)